In [1]:
import sys
sys.path.insert(0, 'utils')
import pandas as pd
import matplotlib.pyplot as plt
import ds_charts as ds
from ds_charts import HEIGHT
import numpy as np
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import json
import collections
import datetime
from dateutil.relativedelta import relativedelta
import timeit
from math import radians, cos, sin, asin, sqrt

data_folder = 'data/gccd/scale/'
model_folder = 'data/gccd/'



In [3]:
# Domain Knowledge 
holidays = pd.read_csv('holidays.csv')
holidays['date'] =  pd.to_datetime(holidays['date'],format="%Y-%m-%d")

day_periods = [(1, (datetime.datetime(2000,1,1,0,0,0),  datetime.datetime(2000,1,1,2,59,59))),
           (2, (datetime.datetime(2000,1,1,3,0,0),  datetime.datetime(2000,1,1,5,59,59))),
           (3, (datetime.datetime(2000,1,1,6,0,0),  datetime.datetime(2000,1,1,8,59,59))),
           (4, (datetime.datetime(2000,1,1,9,0,0),  datetime.datetime(2000,1,1,11,59,59))),
           (5, (datetime.datetime(2000,1,1,12,0,0),  datetime.datetime(2000,1,1,14,59,59))),
           (6, (datetime.datetime(2000,1,1,15,0,0),  datetime.datetime(2000,1,1,17,59,59))),
           (7, (datetime.datetime(2000,1,1,18,0,0),  datetime.datetime(2000,1,1,20,59,59))),
           (8, (datetime.datetime(2000,1,1,21,0,0),  datetime.datetime(2000,1,1,23,59,59)))]

energy_prices = [(1, (datetime.datetime(2000,1,1,22,0,0),  datetime.datetime(2000,1,1,23,59,59))),
           (1, (datetime.datetime(2000,1,1,0,0,0),  datetime.datetime(2000,1,1,7,59,59))),
           (2, (datetime.datetime(2000,1,1,8,0,0),  datetime.datetime(2000,1,1,8,59,59))),
           (2, (datetime.datetime(2000,1,1,10,30,0),  datetime.datetime(2000,1,1,17,59,59))),
           (2, (datetime.datetime(2000,1,1,20,30,0),  datetime.datetime(2000,1,1,21,59,59))),
           (3, (datetime.datetime(2000,1,1,9,0,0),  datetime.datetime(2000,1,1,10,29,59))),
           (3, (datetime.datetime(2000,1,1,18,0,0),  datetime.datetime(2000,1,1,20,29,59)))]

seasons = [(1, (datetime.date(2000,  1,  1),  datetime.date(2000,  3, 20))),
           (2, (datetime.date(2000,  3, 21),  datetime.date(2000,  6, 20))),
           (3, (datetime.date(2000,  6, 21),  datetime.date(2000,  9, 22))),
           (4, (datetime.date(2000,  9, 23),  datetime.date(2000, 12, 20))),
           (1, (datetime.date(2000, 12, 21),  datetime.date(2000, 12, 31)))]

center_baltimore = (39.30746849825375, -76.61560625253648)

# crime_type = [
#     (0, ['AUTO THEFT','LARCENY FROM AUTO','LARCENY']),
#     (1, ['ROBBERY - STREET', 'ROBBERY - COMMERCIAL','ROBBERY - CARJACKING','BURGLARY','ROBBERY - RESIDENCE']),
#     (2, ['COMMON ASSAULT']),
#     (3, ['AGG. ASSAULT','ASSAULT BY THREAT']),
#     (4, ['ARSON']),
#     (5, ['RAPE','SHOOTING']),
#     (6, ['HOMICIDE']),
# ]

crime_type = [
    (1, 'LARCENY'), (2,'LARCENY FROM AUTO'), (3,'AUTO THEFT'), (11,'ROBBERY - STREET'), (12,'ROBBERY - COMMERCIAL'), (13,'ROBBERY - CARJACKING'),(14,'ROBBERY - RESIDENCE'),(15,'BURGLARY'),(21,'COMMON ASSAULT'),(31,'ASSAULT BY THREAT'),(32,'AGG. ASSAULT'),(41,'ARSON'),(51,'SHOOTING'),(52,'RAPE'),(61,'HOMICIDE')
]
weapon_type = [
    (0, 'NONE'), (1,'HANDS'), (2,'OTHER'), (3,'KNIFE'), (4,'FIREARM')
]

# Kept data for operations
temp_data = None


def generate_variables(model,loop_dataset,edit_dataset):
    global temp_data
    relations_queue = model['relations']

    while len(relations_queue) != 0:
        current_relation = relations_queue[0]
        print(current_relation['name'])
        newvar_name = current_relation['output']
        inputs = current_relation['inputs']
        groupby = current_relation['groupby']
        if set(inputs).issubset(loop_dataset.columns):         # if inputs already exist in the dataset
            start_time = timeit.default_timer()
            # if len(current_relation['constraint']) == 0:
            #     constraint = "index == index"
            # else:
            constraint = current_relation['constraint']
            if len(current_relation['constraint']) != 0:
                constraint = "row." + constraint
            else:
                constraint = "True"
            if len(groupby) == 0: #LAMBDA: no row dependence
                for index, op in enumerate(current_relation['operations']):
                    if index == 0:
                        edit_dataset[newvar_name] = edit_dataset.apply(lambda row: get_operation(op,*zip(inputs,row[inputs])) if pd.eval(constraint, target = row) else np.nan, axis = 1)
                    else:
                        edit_dataset[newvar_name] = edit_dataset.apply(lambda row: get_operation(op,zip(newvar_name,row[newvar_name])) if pd.eval(constraint, target = row) else np.nan, axis = 1)
            else:
                needed_rows = current_relation['needsRows']
                for i in range(len(loop_dataset)):
                    # counter += 1
                    row = loop_dataset.loc[i]  # we pass the list to get a DataFrame instead of Series
                    if len(groupby) != 0:
                        if needed_rows == 'all':
                            temp_data = loop_dataset[loop_dataset[groupby] == row.loc[groupby]]
                        elif needed_rows < 0:
                            temp_data = loop_dataset[loop_dataset[groupby] == row.loc[groupby]].loc[i + needed_rows+1:i]
                        else:
                            temp_data = loop_dataset[loop_dataset[groupby] == row.loc[groupby]].loc[i:i + needed_rows-1] # get necessary rows to temp_data
                    # else:
                    #     if needed_rows < 0: 
                    #         temp_data = dataset.loc[i+ needed_rows:i]
                    #     else:    
                    #         temp_data = dataset.loc[i:i + needed_rows]
                    if pd.eval(constraint, target = row) == False:
                        edit_dataset.loc[i,newvar_name] = np.nan
                        continue
                    else:
                        for opIndex, op in enumerate(current_relation['operations']):
                            input_values =  [row.loc[inp] for inp in inputs]
                            if opIndex == 0:
                                edit_dataset.loc[i,newvar_name] = get_operation(op,*zip(inputs,input_values))
                            else:
                                edit_dataset.loc[i,newvar_name] = get_operation(op,zip(newvar_name,edit_dataset.loc[i,newvar_name]))
            relations_queue = relations_queue[1:]
            loop_dataset = edit_dataset.copy(deep=True)
            print(timeit.default_timer() - start_time)
        else:
            # send to the bottom of the queue
            relations_queue.append(relations_queue.pop(relations_queue.index(current_relation)))
        
def get_operation(code,*values):
    if code == '+':
        sum_values = [values[i][1] for i,x in enumerate(values)]
        return np.sum(sum_values)
    elif code == 'positive_sum':
        sum_values = [values[i][1] for i,x in enumerate(values) if values[i][1] >= 0]
        return np.sum(sum_values)
    elif code == 'negative_sum':
        sum_values = [values[i][1] for i,x in enumerate(values) if values[i][1] <= 0]
        return np.sum(sum_values)
    elif code == '-':
        return values[0][1] - values[1][1]
    elif code == '*':
        prod_values = [values[i][1] for i,x in enumerate(values)]
        return np.prod(prod_values)
    elif code == '/':
        return round(values[0][1] / values[1][1],2)
    elif code == '>=':
        return values[0][1] >= values[1][1]
    elif code == 'datediff':
        return relativedelta(values[0][1],values[1][1])
    elif code == 'years':
        return values[0][1].years
    elif code == 'months':
        return values[0][1].years * 12 + values[0][1].months
    elif code == 'getHour':
        return values[0][1].hour
    elif code == 'getDay':
        return values[0][1].day
    elif code == 'getMonth':
        return values[0][1].month
    elif code == 'getYear':
        return values[0][1].year
    elif code == 'getWeekday':
        return values[0][1].dayofweek
    elif code == 'getSeason':
        return getSeason(values[0][1],seasons)
    elif code == 'getDayPeriod':
        return getDayPeriod(values[0][1],day_periods)
    elif code == 'getEnergyPrice':
        return getDayPeriod(values[0][1],energy_prices)
    elif code == 'getHoliday':
        return generateHoliday(values[0][1],values[1][1],holidays)
    elif code == 'divide_by_30':
        return values[0][1] / 30
    elif code == 'getAverage':
        return generateAverage(values[0][0])
    elif code == 'getMax':
        return generateMax(values[0][0])
    elif code == 'getMin':
        return generateMin(values[0][0])
    elif code == 'getStd':
        return generateStd(values[0][0])
    elif code == 'generateAvg_2weeks':
        return generateAverage2Weeks(values[0][1],values[1][1])
    elif code == 'generateAvg_2w_100k':
        return values[0][1] * 100000 / values[1][1]
    elif code == 'generateSum_2weeks':
        return generateCumulative2Weeks(values[0][1],values[1][1])
    elif code == 'generateHighRisk_2weeks':
        return generateHighRisk(values[0][1],values[1][1],14)
    elif code == 'getLastYearTemp':
        return generateLastYearTemp(values[0][1],values[1][1],12)
    elif code == 'generateSum_2w_100k':
        return values[0][1] * 100000 / values[1][1]
    elif code == 'generateEstadio_8ed':
        return generateFromTable(values[0][1],values[1][1],estadio)
    elif code == 'generateN_8ed':
        return generateN8ed(values[0][1])
    elif code == 'getDistanceBaltimore':
        return haversine(values[0][1],values[1][1],center_baltimore[0],center_baltimore[1])
    elif code == 'getCrimeType':
        return [x[0] for x in crime_type if x[1] == values[0][1]][0]
    elif code == 'getWeapon':
        return [x[0] for x in weapon_type if x[1] == values[0][1]][0]
    elif code == 'getCases100k':
        return values[0][1] * 100000 / values[1][1]
    elif code == 'getCurrentRisk':
        return values[0][1] > 120
    elif code == 'getAverageDiffPos':
        return values[0][1] >= 0
    elif code == 'generatePM25_safe':
        return values[0][1] >= 35
    elif code == 'generatePM10_safe':
        return values[0][1] >= 150
    elif code == 'generateSO2_safe':
        return values[0][1] >= 0.14
    else:
        return lambda *x : x



def haversine(lat1, lon1, lat2, lon2):
      R = 6372.8
      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)
      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))
      return R * c

def generateFromTable(t,n,table):
    return table[t][n]

def generateN8ed(gg_p):
    if gg_p == 0:
        return 1
    elif gg_p >= 1 and gg_p <= 2:
        return 2
    elif gg_p >= 3 and gg_p <= 6:
        return 3
    elif gg_p >= 7 and gg_p <= 15:
        return 4
    else:
        return 5

def generateAverage(column):
    return np.mean(temp_data[column].to_list())
def generateMax(column):
    return np.max(temp_data[column].to_list())
def generateMin(column):
    return np.min(temp_data[column].to_list())
def generateStd(column):
    return np.std(temp_data[column].to_list())

def getSeason(date,season_dict):
    date = date.replace(year=2000)
    return next(season for season, (start, end) in season_dict
                if start <= date <= end)
def getDayPeriod(date,season_dict):
    date = date.replace(year=2000,month=1,day=1)
    return next(season for season, (start, end) in season_dict
                if start <= date <= end)

def generateAverage2Weeks(date,cases):
    lastCases = temp_data['cases'].to_list()
    return np.mean(lastCases)

def generateHighRisk(date,sum_2weeks,offset):
    high_risk_day = date + datetime.timedelta(days=offset)
    high_risk_cumulative = temp_data[temp_data['current_date'] == high_risk_day]
    if len(high_risk_cumulative) == 0:
        return np.nan
    else:
        return high_risk_cumulative.iloc[0]['sum_2w_100k'] >= 120.0

def generateLastYearTemp(date,temp,offset):
    last_year_day = date - relativedelta(months=offset)
    last_year_record = temp_data[temp_data['current_date'] == last_year_day]
    if len(last_year_record) == 0:
        return np.nan
    else:
        return last_year_record.iloc[0]['temperature']

def generateCumulative2Weeks(date,cases):
    lastCases = temp_data['cases'].to_list()
    return np.sum(lastCases)

def generateHoliday(date,country,holiday_data):
    return holiday_data.loc[(holiday_data['date'] == date) & (holiday_data['country'] == country)].any().all()

# Get dataset
for i in range(6,9):
    print(i)
    start_time = timeit.default_timer()
    data = pd.read_csv(f'{data_folder}gccd_scale_{i}.csv',parse_dates=True, infer_datetime_format=True)
    print('read csv')
    print(timeit.default_timer() - start_time)

    # Getting ER Model of case study data
    f = open(f'{model_folder}gccd_model.json')
    model = json.load(f)

    entity_name_list = [entity['name'] for entity in model['entities']]
    # Raise Error if entities do not match columns:
    if collections.Counter(entity_name_list) != collections.Counter(data.columns.to_list()):
        raise ValueError("Entities and Columns do not have the same size.")


    not_observed = [entity['name'] for entity in model['entities'] if not entity['observed']]
    created_vars = [relation['output'] for relation in model['relations']]
    date_columns = [entity['name'] for entity in model['entities'] if entity['type'] == 'datetime']
    data['current_date'] = pd.to_datetime(data['current_date'], format = "%Y-%m-%d %H:%M:%S")
    data_observed = data.drop(columns = not_observed)

    data_loop = data_observed.copy(deep=True)
    start_time = timeit.default_timer()
    generate_variables(model,data_loop,data_observed)
    start_time = timeit.default_timer()
    data_observed.to_csv(f'{data_folder}gccd_scale_{i}_gen.csv',index=False)
    print('save csv')
    print(timeit.default_timer() - start_time)

6
read csv
0.1283506000036141
day
74.0037755999947
month
69.8407645000043
year
68.68519699999888
season
77.00625589999981
country_lat
1403.6301183000032
country_lon
1310.8209181999991
avg_temperature


KeyboardInterrupt: 